### k-전체세대수 결측값 채우기

In [ ]:
# 1. 아파트명 → 세대수 매핑 딕셔너리 생성
apt_to_units = base_df.groupby('아파트명')['k-전체세대수'].first().to_dict()
# 2. 결측값 채우기
base_df['k-전체세대수'] = base_df['k-전체세대수'].fillna(base_df['아파트명'].map(apt_to_units))


In [ ]:
# 남은 결측값 확인
mask_missing = base_df['k-전체세대수'].isna()

# 구 기준 평균으로 채우기
dong_mean = base_df.groupby('동')['k-전체세대수'].transform('mean')
base_df.loc[mask_missing, 'k-전체세대수'] = dong_mean[mask_missing]

In [ ]:
# 그래도 남으면 전체 중앙값으로
base_df['k-전체세대수'] = base_df['k-전체세대수'].fillna(base_df['k-전체세대수'].median())

### k-전체동수 채우기

In [ ]:
base_df['k-전체동수'] = base_df.groupby(['시군구', '아파트명'])['k-전체동수'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

In [ ]:
# 1. 기존 데이터에서 동당 평균 세대 수 계산 (결측치 없는 행 기준)
valid = base_df[base_df['k-전체동수'].notnull() & base_df['k-전체세대수'].notnull()]

# 동당 평균 세대 수
avg_households_per_building = (valid['k-전체세대수'] / valid['k-전체동수']).mean()

print(f"동당 평균 세대 수 (계산 기준): {avg_households_per_building:.2f}")

# 2. 결측치 대체: k-전체세대수를 기반으로 추정된 전체동수로 대체
def estimate_total_buildings(row):
    if pd.isnull(row['k-전체동수']) and pd.notnull(row['k-전체세대수']):
        return round(row['k-전체세대수'] / avg_households_per_building)
    return row['k-전체동수']

base_df['k-전체동수'] = base_df.apply(estimate_total_buildings, axis=1)

### 원핫 인코딩

In [ ]:
onehot_cols =[ 'k-난방방식','k-세대타입(분양형태)','세대전기계약방법', 'k-단지분류(아파트,주상복합등등)', '기타/의무/임대/임의=1/2/3/4','k-복도유형']

In [ ]:
def one_hot_encode_train_test(train_df, test_df, columns):
    # train과 test 합치기 (인코딩 컬럼 구조 통일을 위해)
    train_df['__is_train__'] = 1
    test_df['__is_train__'] = 0
    combined = pd.concat([train_df, test_df], axis=0)

    # get_dummies로 인코딩
    combined_encoded = pd.get_dummies(combined, columns=columns, drop_first=True)

    # 다시 분리
    train_encoded = combined_encoded[combined_encoded['__is_train__'] == 1].drop(columns='__is_train__')
    test_encoded = combined_encoded[combined_encoded['__is_train__'] == 0].drop(columns='__is_train__')

    return train_encoded, test_encoded

dt_train_encoded, dt_test_encoded = one_hot_encode_train_test(dt_train.copy(), dt_test.copy(), onehot_cols)

In [ ]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.= 
# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}
label_encoding_cols = [col for col in categorical_columns_v2 if col not in onehot_cols]

# Implement Label Encoding
for col in tqdm( label_encoding_cols ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train_encoded[col].astype(str) )
    dt_train_encoded[col] = lbl.transform(dt_train_encoded[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(dt_test_encoded[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    dt_test_encoded[col] = lbl.transform(dt_test_encoded[col].astype(str))

100%|██████████| 5/5 [00:01<00:00,  3.73it/s]
